In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [129]:
from datetime import datetime

In [88]:
df = pd.read_csv('moviedf.csv')

In [6]:
print df.columns

Index([u'title', u'dtg', u'dist', u'release', u'genre', u'runtime', u'budget',
       u'rating', u'foreign', u'foreign_dist', u'foreign_release',
       u'foreign_opening', u'foreign_pct', u'foreign_total', u'foreign_asof',
       u'hungary', u'hun_dist', u'hun_release', u'hun_opening', u'hun_pct',
       u'hun_total', u'hun_asof'],
      dtype='object')


In [8]:
column_meanings = {
    'title': 'movie title',
    'dtg': 'domestic total gross',
    'dist': 'domestic distributor',
    'release': 'domestic release',
    'genre': 'genre', 
    'runtime': 'runtime',
    'budget': 'production budget',
    'rating': 'MPAA rating',
    'foreign': 'FOREIGN TOTAL (can drop this table)',
    'foreign_dist': 'foreign dist (can drop this table)',
    'foreign_release': 'earliest foreign release',
    'foreign_opening': 'gross opening weekend, all foreign',
    'foreign_pct': '% opening weekend of total, all foreign',
    'foreign_total': 'total foreign gross',
    'foreign_asof': 'date total gross calculated to',
    'hungary': 'hungary (can drop this table)',
    'hungary': 'hungarian distributor',
    'hun_release': 'hungarian release',
    'hun_opening': 'gross opening weekend, hungary',
    'hun_pct': '% opening weekend of hungarian total',
    'hun_total': 'total hungarian gross',
    'hun_asof': 'date hun gross calculated to'
}

In [89]:
df = df.drop('foreign', axis=1)

In [90]:
df = df.drop('hungary', axis=1)

In [91]:
df = df.drop('foreign_dist', axis=1)

In [16]:
del(column_meanings['foreign'])

In [18]:
del(column_meanings['hungary'])
del(column_meanings['foreign_dist'])

In [77]:
df.head()

,title,dtg,dist,release,genre,runtime,budget,rating,foreign_release,foreign_opening,foreign_pct,foreign_total,foreign_asof,hun_dist,hun_release,hun_opening,hun_pct,hun_total,hun_asof
0,Star Wars: The Force Awakens,NaN,Buena Vista,"December 18, 2015",Sci-Fi Fantasy,2 hrs. 16 min.,$200 million,PG-13,12/16/15,"$281,000,000",26.5%,"$1,060,700,000",1/24/16,Forum Films,12/18/15,"$1,508,298",24.4%,"$6,181,662",1/24/16
1,Jurassic World,"$652,270,625",Universal,"June 12, 2015",Sci-Fi Horror,2 hrs. 4 min.,$150 million,PG-13,6/11/15,"$316,102,740",31.1%,"$1,016,714,301",11/11/15,UIP,6/11/15,"$604,551",29.5%,"$2,051,915",7/5/15
2,Avengers: Age of Ultron,"$459,005,868",Buena Vista,"May 1, 2015",Action / Adventure,2 hrs. 21 min.,$250 million,PG-13,4/22/15,"$201,200,000",21.3%,"$946,029,899",NaN,Forum Films,5/7/15,"$807,040",40.7%,"$1,980,532",6/21/15
3,Inside Out,"$356,461,711",Buena Vista,"June 19, 2015",Animation,1 hrs. 34 min.,$175 million,PG,6/17/15,n/a,-,"$499,668,421",12/29/15,Forum Films,6/25/15,"$296,617",24.3%,"$1,218,401",9/27/15
4,Furious 7,"$353,007,020",Universal,"April 3, 2015",Action,2 hrs. 17 min.,$190 million,PG-13,4/1/15,"$250,538,637",21.6%,"$1,162,040,651",10/28/15,UIP,4/2/15,"$661,897",33.8%,"$1,958,035",5/24/15


In [25]:
correlation = df.corr(method='pearson', min_periods=1)

In [92]:
converted = df.copy()

In [97]:
converted['dtg'] = df['dtg'].apply(lambda x: str(x))

In [101]:
converted['dtg'] = converted['dtg'].apply(lambda x: x.replace('$', '').replace(',',''))
converted['dtg'] = converted['dtg'].apply(lambda x: np.nan if x=='nan' else int(x))

In [103]:
converted['foreign_opening'] = converted['foreign_opening'].apply(lambda x: str(x).replace('$', '').replace(',','')
converted['foreign_opening'] = converted['foreign_opening'].apply(lambda x: np.nan if x=='n/a' else int(x))

In [127]:
converted['title'] = converted['title'].apply(lambda x: str(x))
converted['dist'] = converted['dist'].apply(lambda x: str(x))
converted['genre'] = converted['genre'].apply(lambda x: str(x))
converted['rating'] = converted['rating'].apply(lambda x: str(x))
converted['hun_dist'] = converted['hun_dist'].apply(lambda x: str(x))

In [121]:
converted['foreign_total'] = converted['foreign_total'].apply(lambda x: str(x).replace('$', '').replace(',',''))
converted['foreign_total'] = converted['foreign_total'].apply(lambda x: np.nan if x=='n/a' else int(x))

In [125]:
converted['hun_opening'] = converted['hun_opening'].apply(lambda x: str(x).replace('$', '').replace(',',''))
converted['hun_opening'] = converted['hun_opening'].apply(lambda x: np.nan if x=='n/a' or x=='-' else int(x))
converted['hun_total'] = converted['hun_total'].apply(lambda x: str(x).replace('$', '').replace(',',''))
converted['hun_total'] = converted['hun_total'].apply(lambda x: np.nan if x=='n/a' else int(x))

In [128]:
converted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 543 entries, 0 to 542
Data columns (total 19 columns):
title              543 non-null object
dtg                525 non-null float64
dist               543 non-null object
release            543 non-null object
genre              543 non-null object
runtime            543 non-null object
budget             485 non-null object
rating             543 non-null object
foreign_release    538 non-null object
foreign_opening    424 non-null float64
foreign_pct        543 non-null object
foreign_total      539 non-null float64
foreign_asof       474 non-null object
hun_dist           543 non-null object
hun_release        543 non-null object
hun_opening        528 non-null float64
hun_pct            543 non-null object
hun_total          538 non-null float64
hun_asof           538 non-null object
dtypes: float64(5), object(14)
memory usage: 84.8+ KB


In [116]:
converted.head()

,title,dtg,dist,release,genre,runtime,budget,rating,foreign_release,foreign_opening,foreign_pct,foreign_total,foreign_asof,hun_dist,hun_release,hun_opening,hun_pct,hun_total,hun_asof
0,Star Wars: The Force Awakens,NaN,Buena Vista,"December 18, 2015",Sci-Fi Fantasy,2 hrs. 16 min.,$200 million,PG-13,12/16/15,281000000,26.5%,"$1,060,700,000",1/24/16,Forum Films,12/18/15,"$1,508,298",24.4%,"$6,181,662",1/24/16
1,Jurassic World,652270625,Universal,"June 12, 2015",Sci-Fi Horror,2 hrs. 4 min.,$150 million,PG-13,6/11/15,316102740,31.1%,"$1,016,714,301",11/11/15,UIP,6/11/15,"$604,551",29.5%,"$2,051,915",7/5/15
2,Avengers: Age of Ultron,459005868,Buena Vista,"May 1, 2015",Action / Adventure,2 hrs. 21 min.,$250 million,PG-13,4/22/15,201200000,21.3%,"$946,029,899",NaN,Forum Films,5/7/15,"$807,040",40.7%,"$1,980,532",6/21/15
3,Inside Out,356461711,Buena Vista,"June 19, 2015",Animation,1 hrs. 34 min.,$175 million,PG,6/17/15,NaN,-,"$499,668,421",12/29/15,Forum Films,6/25/15,"$296,617",24.3%,"$1,218,401",9/27/15
4,Furious 7,353007020,Universal,"April 3, 2015",Action,2 hrs. 17 min.,$190 million,PG-13,4/1/15,250538637,21.6%,"$1,162,040,651",10/28/15,UIP,4/2/15,"$661,897",33.8%,"$1,958,035",5/24/15


In [130]:
converted['release'] = pd.to_datetime(converted['release'], errors='coerce', infer_datetime_format=True)

In [133]:
converted['foreign_release'] = pd.to_datetime(converted['foreign_release'], errors='coerce', infer_datetime_format=True)
converted['hun_release'] = pd.to_datetime(converted['hun_release'], errors='coerce', infer_datetime_format=True)
converted['foreign_asof'] = pd.to_datetime(converted['foreign_asof'], errors='coerce', infer_datetime_format=True)
converted['hun_asof'] = pd.to_datetime(converted['hun_asof'], errors='coerce', infer_datetime_format=True)

In [139]:
converted['foreign_pct'] = converted['foreign_pct'].apply(lambda x: str(x).replace('%', ''))

In [141]:
converted.head()

,title,dtg,dist,release,genre,runtime,budget,rating,foreign_release,foreign_opening,foreign_pct,foreign_total,foreign_asof,hun_dist,hun_release,hun_opening,hun_pct,hun_total,hun_asof
0,Star Wars: The Force Awakens,NaN,Buena Vista,2015-12-18,Sci-Fi Fantasy,2 hrs. 16 min.,$200 million,PG-13,2015-12-16,281000000,26.5,1060700000,2016-01-24,Forum Films,2015-12-18,1508298,24.4%,6181662,1/24/16
1,Jurassic World,652270625,Universal,2015-06-12,Sci-Fi Horror,2 hrs. 4 min.,$150 million,PG-13,2015-06-11,316102740,31.1,1016714301,2015-11-11,UIP,2015-06-11,604551,29.5%,2051915,7/5/15
2,Avengers: Age of Ultron,459005868,Buena Vista,2015-05-01,Action / Adventure,2 hrs. 21 min.,$250 million,PG-13,2015-04-22,201200000,21.3,946029899,NaT,Forum Films,2015-05-07,807040,40.7%,1980532,6/21/15
3,Inside Out,356461711,Buena Vista,2015-06-19,Animation,1 hrs. 34 min.,$175 million,PG,2015-06-17,NaN,-,499668421,2015-12-29,Forum Films,2015-06-25,296617,24.3%,1218401,9/27/15
4,Furious 7,353007020,Universal,2015-04-03,Action,2 hrs. 17 min.,$190 million,PG-13,2015-04-01,250538637,21.6,1162040651,2015-10-28,UIP,2015-04-02,661897,33.8%,1958035,5/24/15


In [149]:
converted['foreign_pct'] = converted['foreign_pct'].apply(lambda x: np.nan if x=='-' else float(x)/100)

In [151]:
converted['hun_pct'] = converted['hun_pct'].apply(lambda x: str(x).replace('%', ''))
converted['hun_pct'] = converted['hun_pct'].apply(lambda x: np.nan if x=='-' else float(x)/100)

In [152]:
converted.head()

,title,dtg,dist,release,genre,runtime,budget,rating,foreign_release,foreign_opening,foreign_pct,foreign_total,foreign_asof,hun_dist,hun_release,hun_opening,hun_pct,hun_total,hun_asof
0,Star Wars: The Force Awakens,NaN,Buena Vista,2015-12-18,Sci-Fi Fantasy,2 hrs. 16 min.,$200 million,PG-13,2015-12-16,281000000,0.265,1060700000,2016-01-24,Forum Films,2015-12-18,1508298,0.244,6181662,1/24/16
1,Jurassic World,652270625,Universal,2015-06-12,Sci-Fi Horror,2 hrs. 4 min.,$150 million,PG-13,2015-06-11,316102740,0.311,1016714301,2015-11-11,UIP,2015-06-11,604551,0.295,2051915,7/5/15
2,Avengers: Age of Ultron,459005868,Buena Vista,2015-05-01,Action / Adventure,2 hrs. 21 min.,$250 million,PG-13,2015-04-22,201200000,0.213,946029899,NaT,Forum Films,2015-05-07,807040,0.407,1980532,6/21/15
3,Inside Out,356461711,Buena Vista,2015-06-19,Animation,1 hrs. 34 min.,$175 million,PG,2015-06-17,NaN,NaN,499668421,2015-12-29,Forum Films,2015-06-25,296617,0.243,1218401,9/27/15
4,Furious 7,353007020,Universal,2015-04-03,Action,2 hrs. 17 min.,$190 million,PG-13,2015-04-01,250538637,0.216,1162040651,2015-10-28,UIP,2015-04-02,661897,0.338,1958035,5/24/15


In [154]:
converted['hun_asof'] = pd.to_datetime(converted['hun_asof'], errors='coerce', infer_datetime_format=True)

In [156]:
converted.head()

,title,dtg,dist,release,genre,runtime,budget,rating,foreign_release,foreign_opening,foreign_pct,foreign_total,foreign_asof,hun_dist,hun_release,hun_opening,hun_pct,hun_total,hun_asof
0,Star Wars: The Force Awakens,NaN,Buena Vista,2015-12-18,Sci-Fi Fantasy,2 hrs. 16 min.,$200 million,PG-13,2015-12-16,281000000,0.265,1060700000,2016-01-24,Forum Films,2015-12-18,1508298,0.244,6181662,1/24/16
1,Jurassic World,652270625,Universal,2015-06-12,Sci-Fi Horror,2 hrs. 4 min.,$150 million,PG-13,2015-06-11,316102740,0.311,1016714301,2015-11-11,UIP,2015-06-11,604551,0.295,2051915,7/5/15
2,Avengers: Age of Ultron,459005868,Buena Vista,2015-05-01,Action / Adventure,2 hrs. 21 min.,$250 million,PG-13,2015-04-22,201200000,0.213,946029899,NaT,Forum Films,2015-05-07,807040,0.407,1980532,6/21/15
3,Inside Out,356461711,Buena Vista,2015-06-19,Animation,1 hrs. 34 min.,$175 million,PG,2015-06-17,NaN,NaN,499668421,2015-12-29,Forum Films,2015-06-25,296617,0.243,1218401,9/27/15
4,Furious 7,353007020,Universal,2015-04-03,Action,2 hrs. 17 min.,$190 million,PG-13,2015-04-01,250538637,0.216,1162040651,2015-10-28,UIP,2015-04-02,661897,0.338,1958035,5/24/15


In [162]:
test = pd.concat([converted['foreign_release'], converted['foreign_opening']], axis=1)

In [164]:
numerical = pd.concat([
        converted['dtg'],
        converted['release'],
        converted['foreign_release'],
        converted['foreign_opening'],
        converted['foreign_pct'],
        converted['foreign_total'],
        converted['hun_release'],
        converted['hun_opening'],
        converted['hun_pct'],
        converted['hun_total']
    ],
                      axis=1)

In [167]:
correlation = numerical.corr()

In [168]:
correlation

,dtg,foreign_opening,foreign_pct,foreign_total,hun_opening,hun_pct,hun_total
dtg,1.000000,0.661372,0.246170,0.850733,0.665883,-0.202101,0.698851
foreign_opening,0.661372,1.000000,0.653276,0.772540,0.794672,-0.107985,0.662716
foreign_pct,0.246170,0.653276,1.000000,0.287926,0.417173,0.043969,0.240962
foreign_total,0.850733,0.772540,0.287926,1.000000,0.796956,-0.216934,0.848403
hun_opening,0.665883,0.794672,0.417173,0.796956,1.000000,-0.186447,0.873483
hun_pct,-0.202101,-0.107985,0.043969,-0.216934,-0.186447,1.000000,-0.313306
hun_total,0.698851,0.662716,0.240962,0.848403,0.873483,-0.313306,1.000000


Q: How many hungarian and american distributors are there? Should I create dummy variables or bucket them?

In [169]:
len(converted['hun_dist'].unique())

20

In [170]:
len(converted['dist'].unique())

25

Q: Is the distributor ever the same in hungary and in the US?

In [171]:
converted['same_dist'] = converted['dist']==converted['hun_dist']

In [174]:
converted[converted['same_dist']==True]

,title,dtg,dist,release,genre,runtime,budget,rating,foreign_release,foreign_opening,foreign_pct,foreign_total,foreign_asof,hun_dist,hun_release,hun_opening,hun_pct,hun_total,hun_asof,same_dist
352,We Bought a Zoo,75624550,Fox,2011-12-23,Comedy / Drama,2 hrs. 4 min.,NaN,PG,2011-12-22,998024,0.022,44457291,2012-07-01,Fox,2012-03-22,27868,0.206,135079,4/22/12,True
470,Avatar,749766139,Fox,2009-12-18,Sci-Fi Adventure,2 hrs. 40 min.,NaN,PG-13,2009-12-16,164545565,0.081,2027457462,NaT,Fox,2009-12-17,713220,0.098,7300532,6/13/10,True
482,Night at the Museum: Battle of the Smithsonian,177243721,Fox,2009-05-22,Family Adventure,1 hrs. 45 min.,$150 million,PG,2009-05-20,49036322,0.208,235862449,2010-03-09,Fox,2009-05-21,112991,0.190,593303,7/19/09,True


In [175]:
pd.Timedelta('2 hours 4 minutes')

Timedelta('0 days 02:04:00')

In [176]:
converted['runtime'] = converted['runtime'].apply(lambda x: x.replace('.', '').replace('hrs','hours').replace('min', 'minutes'))

In [180]:
converted['runtime'] = converted['runtime'].apply(lambda x: pd.Timedelta(x))

In [195]:
converted['budget'] = converted['budget'].apply(lambda x: np.nan if x=='nan' else int(x))

In [197]:
converted.head()

,title,dtg,dist,release,genre,runtime,budget,rating,foreign_release,foreign_opening,foreign_pct,foreign_total,foreign_asof,hun_dist,hun_release,hun_opening,hun_pct,hun_total,hun_asof,same_dist
0,Star Wars: The Force Awakens,NaN,Buena Vista,2015-12-18,Sci-Fi Fantasy,02:16:00,200000000,PG-13,2015-12-16,281000000,0.265,1060700000,2016-01-24,Forum Films,2015-12-18,1508298,0.244,6181662,1/24/16,False
1,Jurassic World,652270625,Universal,2015-06-12,Sci-Fi Horror,02:04:00,150000000,PG-13,2015-06-11,316102740,0.311,1016714301,2015-11-11,UIP,2015-06-11,604551,0.295,2051915,7/5/15,False
2,Avengers: Age of Ultron,459005868,Buena Vista,2015-05-01,Action / Adventure,02:21:00,250000000,PG-13,2015-04-22,201200000,0.213,946029899,NaT,Forum Films,2015-05-07,807040,0.407,1980532,6/21/15,False
3,Inside Out,356461711,Buena Vista,2015-06-19,Animation,01:34:00,175000000,PG,2015-06-17,NaN,NaN,499668421,2015-12-29,Forum Films,2015-06-25,296617,0.243,1218401,9/27/15,False
4,Furious 7,353007020,Universal,2015-04-03,Action,02:17:00,190000000,PG-13,2015-04-01,250538637,0.216,1162040651,2015-10-28,UIP,2015-04-02,661897,0.338,1958035,5/24/15,False


In [198]:
converted['y_values'] = converted['hun_total']/converted['foreign_total']

In [199]:
factors = converted.copy()

In [206]:
factors = factors.drop('foreign_opening', axis=1)

In [207]:
factors = factors.drop('foreign_pct', axis=1)
factors = factors.drop('foreign_total', axis=1)
factors = factors.drop('foreign_asof', axis=1)

In [209]:
factors['hun_releasedelta'] = factors['hun_release']-factors['foreign_release']

In [211]:
factors['foreign_releasedelta'] = factors['foreign_release']-factors['release']

In [214]:
factors['genre'].unique()

array(['Sci-Fi Fantasy', 'Sci-Fi Horror', 'Action / Adventure',
       'Animation', 'Action', 'Sci-Fi', 'Fantasy', 'Comedy', 'Romance',
       'Sci-Fi Action', 'Drama', 'Family Comedy', 'Horror Comedy',
       'Family', 'Thriller', 'Comedy / Drama', 'Crime Drama', 'Horror',
       'Western', 'Action Drama', 'Action Fantasy', 'Action Comedy',
       'Sci-Fi Adventure', 'War Drama', 'Family Adventure',
       'Period Action', 'Sci-Fi Thriller', 'Musical', 'War',
       'Action Horror', 'Action Thriller', 'Romantic Comedy',
       'Fantasy Drama', 'Western Comedy', 'Adventure', 'Concert',
       'Historical Drama', 'Sci-Fi Comedy', 'Drama / Thriller',
       'Horror Thriller', 'Music Drama', 'Period Adventure',
       'Sports Drama', 'Period Drama', 'Romantic Thriller',
       'Adventure Comedy', 'Action / Crime', 'Fantasy Comedy',
       'Documentary'], dtype=object)

In [216]:
factors.columns

Index([u'title', u'dtg', u'dist', u'release', u'genre', u'runtime', u'budget',
       u'rating', u'foreign_release', u'hun_dist', u'hun_release',
       u'hun_opening', u'hun_pct', u'hun_total', u'hun_asof', u'same_dist',
       u'y_values', u'hun_releasedelta', u'foreign_releasedelta'],
      dtype='object')

In [219]:
factors = factors[108:]

In [221]:
factors = factors.reset_index()

In [223]:
factors = factors.drop('index', axis=1)

In [224]:
factors

,title,dtg,dist,release,genre,runtime,budget,rating,foreign_release,hun_dist,hun_release,hun_opening,hun_pct,hun_total,hun_asof,same_dist,y_values,hun_releasedelta,foreign_releasedelta
0,The Other Woman,83911193,Fox,2014-04-25,Comedy,01:49:00,40000000,PG-13,2014-04-16,InterCom,2014-04-24,216644,0.187,1157744,7/13/14,False,0.010264,8 days,-9 days
1,Penguins of Madagascar,83350911,Fox,2014-11-26,Animation,01:32:00,132000000,PG,2014-11-14,InterCom,2014-11-27,293683,0.245,1200609,1/25/15,False,0.004145,13 days,-12 days
2,Let's Be Cops,82390774,Fox,2014-08-13,Action Comedy,01:44:00,17000000,R,2014-08-14,InterCom,2014-09-11,148440,0.241,615758,10/26/14,False,0.011028,28 days,1 days
3,The Monuments Men,78031620,Sony / Columbia,2014-02-07,Action Drama,01:50:00,70000000,PG-13,2014-02-12,InterCom,2014-02-20,147144,0.399,368623,3/16/14,False,0.004790,8 days,5 days
4,Hercules,72688614,Paramount,2014-07-25,Action / Adventure,01:38:00,100000000,PG-13,2014-07-23,Forum,2014-07-24,258388,0.381,677457,8/17/14,False,0.003936,1 days,-2 days
5,The Purge: Anarchy,71962800,Universal,2014-07-18,Thriller,01:43:00,9000000,R,2014-07-17,UIP,2014-07-17,14966,1.000,14966,7/20/14,False,0.000374,0 days,-1 days
6,Exodus: Gods and Kings,65014513,Fox,2014-12-12,Drama,02:30:00,140000000,PG-13,2014-12-04,InterCom,2014-12-11,184492,0.284,650465,1/18/15,False,0.003202,7 days,-8 days
7,The Nut Job,64251541,Open Road Films,2014-01-17,Animation,01:26:00,42000000,PG,2014-01-22,Provideo,2014-01-23,135531,0.208,652769,3/16/14,False,0.011526,1 days,5 days
8,Son of God,59700064,Fox,2014-02-28,Drama,02:18:00,NaN,PG-13,2014-04-02,A Company,2014-04-10,22729,0.414,54899,4/20/14,False,0.006778,8 days,33 days
9,The Grand Budapest Hotel,59301324,Fox Searchlight,2014-03-07,Comedy,01:40:00,NaN,R,2014-02-26,InterCom,2014-03-20,90450,0.215,420076,4/27/14,False,0.003637,22 days,-9 days


In [246]:
# big_genre = {
#     'Sci-Fi':   ['Sci-Fi Fantasy',
#                     'Sci-Fi Horror',
#                     'Sci-Fi',
#                     'Sci-Fi Action',
#                     'Sci-Fi Adventure', 
#                     'Sci-Fi Thriller',
#                     'Sci-Fi Comedy'],
#     'Action':   ['Action / Adventure',
#                     'Action',
#                     'Action Drama',
#                     'Period Action',
#                     'Action Horror',
#                     'Action / Crime', 
#                     'Action Fantasy'],
#     'Fantasy':   ['Fantasy'],
#     'Adventure': ['Adventure', 'Western'],
#     'Comedy':    ['Comedy',
#                     'Family Comedy',
#                     'Horror Comedy',
#                     'Comedy / Drama', 
#                     'Action Comedy',
#                     'Romantic Comedy',
#                     'Western Comedy',
#                     'Fantasy Comedy',
#                     'Adventure Comedy'],
#     'Romance':   ['Romance'],
#     'Drama':     ['Drama', 
#                     'Crime Drama', 
#                     'War Drama',
#                     'War',
#                     'Historical Drama',
#                     'Period Drama', 
#                     'Period Adventure',
#                     'Sports Drama',
#                     'Fantasy Drama'],
#     'Family':     ['Family',
#                     'Family Adventure',
#                     'Animation'],
#     'Thriller':    ['Thriller',
#                     'Horror',
#                     'Horror Thriller',
#                     'Romantic Thriller',
#                     'Drama / Thriller',
#                     'Action Thriller'],
#     'Musical':     ['Musical', 
#                     'Music Drama',
#                     'Concert'],
#     'Documentary': ['Documentary']
# }

big_genre = {
    'Sci-Fi Fantasy': 'Sci-Fi',
'Sci-Fi Horror': 'Sci-Fi',
'Sci-Fi': 'Sci-Fi',
'Sci-Fi Action': 'Sci-Fi',
'Sci-Fi Adventure': 'Sci-Fi',
'Sci-Fi Thriller': 'Sci-Fi',
'Sci-Fi Comedy': 'Sci-Fi',

'Action / Adventure': 'Action',
'Action': 'Action',
'Action Drama': 'Action',
'Period Action': 'Action',
'Action Horror': 'Action',
'Action / Crime': 'Action',
'Action Fantasy': 'Action',

'Fantasy' : 'Fantasy',

'Adventure' :'Adventure', 
'Western' : 'Adventure',  

'Comedy': 'Comedy',
'Family Comedy': 'Comedy',
'Horror Comedy': 'Comedy',
'Comedy / Drama': 'Comedy',
'Action Comedy': 'Comedy',
'Romantic Comedy': 'Comedy',
'Western Comedy': 'Comedy',
'Fantasy Comedy': 'Comedy',
'Adventure Comedy': 'Comedy',

'Romance': 'Romance',

'Drama': 'Drama',
'Crime Drama': 'Drama',
'War Drama': 'Drama',
'War': 'Drama',
'Historical Drama': 'Drama',
'Period Drama': 'Drama',
'Period Adventure': 'Drama',
'Sports Drama': 'Drama',
'Fantasy Drama': 'Drama',

'Family': 'Family',
'Family Adventure': 'Family',
'Animation': 'Family',

'Thriller': 'Thriller/Horror',
'Horror': 'Thriller/Horror',
'Horror Thriller': 'Thriller/Horror',
'Romantic Thriller': 'Thriller/Horror',
'Drama / Thriller': 'Thriller/Horror',
'Action Thriller': 'Thriller/Horror',

  
'Musical': 'Music',
'Music Drama': 'Music',
'Concert': 'Music',

'Documentary': 'Documentary'
}

In [247]:
factors['big_genre'] = factors['genre'].apply(lambda x: big_genre[x])

In [249]:
factors['genre'] = factors['big_genre']

In [263]:
factors = factors.drop('big_genre', axis=1)

In [252]:
factors['hun_dist'].unique()

array(['InterCom', 'Forum', 'UIP', 'Provideo', 'A Company', 'Forum Films',
       'Big Bang', 'A', '-', 'WB', 'Fox', 'Budapest', 'PPI', 'UPI',
       'Palace', 'UIP - Duna', "SPI Int'l", 'Sony', 'Intersonic'], dtype=object)

In [260]:
hun_dist_combined = {
    'InterCom': 'InterCom',
    'Forum': 'Forum',
    'Forum Films': 'Forum',
    "SPI Int'l": 'SPI',
    'UIP': 'United',
    'UIP - Duna':'United',
    'UPI': 'Universal',
    'Provideo': 'Provideo',
    'A Company': 'A Company',
    'A': 'A Company',
    'Palace':'Palace',
    '-':'-',
    'WB':'WB',
    'Fox':'Fox',
    'Budapest':'Budapest',
    'Big Bang':'Big Bang',
    'PPI':'Paramount',
    'Sony': 'Sony / Columbia',
    'Intersonic':'Intersonic'
}

In [261]:
factors['hun_dist_combined'] = factors['hun_dist'].apply(lambda x: hun_dist_combined[x])

In [264]:
factors['hun_dist'] = factors['hun_dist_combined']

In [265]:
factors = factors.drop('hun_dist_combined', axis=1)

In [267]:
factors['same_dist'] = factors['dist']==factors['hun_dist']

In [270]:
factors = factors.drop('dtg', axis=1)
factors = factors.drop('hun_total', axis=1)
factors = factors.drop('hun_asof', axis=1)

In [273]:
del factors['dist']
#factors = factors.drop('dist', axis=1)

In [274]:
factors.head()

,title,release,genre,runtime,budget,rating,foreign_release,hun_dist,hun_release,hun_opening,hun_pct,same_dist,y_values,hun_releasedelta,foreign_releasedelta
0,The Other Woman,2014-04-25,Comedy,01:49:00,40000000,PG-13,2014-04-16,InterCom,2014-04-24,216644,0.187,False,0.010264,8 days,-9 days
1,Penguins of Madagascar,2014-11-26,Family,01:32:00,132000000,PG,2014-11-14,InterCom,2014-11-27,293683,0.245,False,0.004145,13 days,-12 days
2,Let's Be Cops,2014-08-13,Comedy,01:44:00,17000000,R,2014-08-14,InterCom,2014-09-11,148440,0.241,False,0.011028,28 days,1 days
3,The Monuments Men,2014-02-07,Action,01:50:00,70000000,PG-13,2014-02-12,InterCom,2014-02-20,147144,0.399,False,0.004790,8 days,5 days
4,Hercules,2014-07-25,Action,01:38:00,100000000,PG-13,2014-07-23,Forum,2014-07-24,258388,0.381,False,0.003936,1 days,-2 days


In [275]:
factors_2 = pd.get_dummies(factors, columns = ['genre', 'hun_dist', 'rating'])

In [276]:
factors_2.head()

,title,release,runtime,budget,foreign_release,hun_release,hun_opening,hun_pct,same_dist,y_values,...,hun_dist_Provideo,hun_dist_SPI,hun_dist_Sony / Columbia,hun_dist_United,hun_dist_Universal,hun_dist_WB,rating_G,rating_PG,rating_PG-13,rating_R
0,The Other Woman,2014-04-25,01:49:00,40000000,2014-04-16,2014-04-24,216644,0.187,False,0.010264,...,0,0,0,0,0,0,0,0,1,0
1,Penguins of Madagascar,2014-11-26,01:32:00,132000000,2014-11-14,2014-11-27,293683,0.245,False,0.004145,...,0,0,0,0,0,0,0,1,0,0
2,Let's Be Cops,2014-08-13,01:44:00,17000000,2014-08-14,2014-09-11,148440,0.241,False,0.011028,...,0,0,0,0,0,0,0,0,0,1
3,The Monuments Men,2014-02-07,01:50:00,70000000,2014-02-12,2014-02-20,147144,0.399,False,0.004790,...,0,0,0,0,0,0,0,0,1,0
4,Hercules,2014-07-25,01:38:00,100000000,2014-07-23,2014-07-24,258388,0.381,False,0.003936,...,0,0,0,0,0,0,0,0,1,0


In [277]:
del factors_2['title']
del factors_2['release']
del factors_2['hun_release']
del factors_2['foreign_release']

In [278]:
factors_2.head()

,runtime,budget,hun_opening,hun_pct,same_dist,y_values,hun_releasedelta,foreign_releasedelta,genre_Action,genre_Adventure,...,hun_dist_Provideo,hun_dist_SPI,hun_dist_Sony / Columbia,hun_dist_United,hun_dist_Universal,hun_dist_WB,rating_G,rating_PG,rating_PG-13,rating_R
0,01:49:00,40000000,216644,0.187,False,0.010264,8 days,-9 days,0,0,...,0,0,0,0,0,0,0,0,1,0
1,01:32:00,132000000,293683,0.245,False,0.004145,13 days,-12 days,0,0,...,0,0,0,0,0,0,0,1,0,0
2,01:44:00,17000000,148440,0.241,False,0.011028,28 days,1 days,0,0,...,0,0,0,0,0,0,0,0,0,1
3,01:50:00,70000000,147144,0.399,False,0.004790,8 days,5 days,1,0,...,0,0,0,0,0,0,0,0,1,0
4,01:38:00,100000000,258388,0.381,False,0.003936,1 days,-2 days,1,0,...,0,0,0,0,0,0,0,0,1,0


In [285]:
factors_2 = factors_2.rename(columns={
        'hun_dist_A Company' : 'hun_dist_ACompany',
        'hun_dist_Big Bang' : 'hun_dist_BigBang',
        'hun_dist_Sony / Columbia' : 'hun_dist_SonyColumbia'
    })

In [307]:
factors_2 = factors_2.rename(columns={
        'genre_Sci-Fi':'genre_SciFi',
        'genre_Thriller/Horror':'genre_ThrillerHorror'
    })

In [324]:
factors_2 = factors_2.rename(columns={
    'rating_PG-13':'rating_PG13'})

In [308]:
features = ['runtime',
 'budget',
 'hun_opening',
 'hun_pct',
 'same_dist',
 'hun_releasedelta',
 'foreign_releasedelta',
 'genre_Action',
 'genre_Adventure',
 'genre_Comedy',
 'genre_Documentary',
 'genre_Drama',
 'genre_Family',
 'genre_Fantasy',
 'genre_Music',
 'genre_Romance',
 'genre_Sci-Fi',
 'genre_Thriller/Horror',
 'hun_dist_-',
 'hun_dist_ACompany',
 'hun_dist_BigBang',
 'hun_dist_Budapest',
 'hun_dist_Forum',
 'hun_dist_Fox',
 'hun_dist_InterCom',
 'hun_dist_Intersonic',
 'hun_dist_Palace',
 'hun_dist_Paramount',
 'hun_dist_Provideo',
 'hun_dist_SPI',
 'hun_dist_SonyColumbia',
 'hun_dist_United',
 'hun_dist_Universal',
 'hun_dist_WB',
 'rating_G',
 'rating_PG',
 'rating_PG-13',
 'rating_R']

In [367]:
import statsmodels.formula.api as sm

In [325]:
model = sm.ols('y_values ~  budget + same_dist + genre_Action + genre_Adventure + genre_Comedy + genre_Drama + genre_Family + genre_Fantasy+ genre_Music + genre_Romance + genre_SciFi + genre_ThrillerHorror + hun_dist_ACompany + hun_dist_BigBang + hun_dist_Budapest + hun_dist_Forum + hun_dist_Fox + hun_dist_InterCom + hun_dist_Intersonic + hun_dist_Palace + hun_dist_Paramount + hun_dist_Provideo + hun_dist_SPI + hun_dist_SonyColumbia + hun_dist_United + hun_dist_Universal + hun_dist_WB + rating_G + rating_PG + rating_PG13', data=factors_2)

In [311]:
#runtime + hun_opening + hun_releasedelta + foreign_releasedelta

In [368]:
model = sm.ols('y_values ~  budget + same_dist + genre_Action + genre_Adventure \
            + genre_Comedy + genre_Drama + genre_Family + genre_Fantasy+ genre_Music \
            + genre_Romance + genre_SciFi + genre_ThrillerHorror + hun_dist_ACompany \
            + hun_dist_BigBang + hun_dist_Budapest + hun_dist_Forum + hun_dist_Fox \
            + hun_dist_InterCom + hun_dist_Intersonic + hun_dist_Palace + hun_dist_Paramount \
            + hun_dist_Provideo + hun_dist_SPI + hun_dist_SonyColumbia + hun_dist_United \
            + hun_dist_Universal + hun_dist_WB + rating_G + rating_PG + rating_PG13', data=factors_2)
results = model.fit()
print('Parameters: ', results.params)
print('R2: ', results.rsquared)
print(results.summary())

('Parameters: ', Intercept                4.358141e-03
same_dist[T.True]        7.620425e-04
budget                  -1.592969e-12
genre_Action            -4.572615e-04
genre_Adventure          2.734136e-04
genre_Comedy             3.132466e-03
genre_Drama             -3.636980e-04
genre_Family             2.402294e-03
genre_Fantasy            1.282473e-03
genre_Music             -5.060808e-04
genre_Romance           -1.335139e-04
genre_SciFi             -2.673704e-04
genre_ThrillerHorror    -1.004582e-03
hun_dist_ACompany        8.291062e-03
hun_dist_BigBang         7.006546e-04
hun_dist_Budapest       -1.279254e-03
hun_dist_Forum          -9.839385e-04
hun_dist_Fox            -2.702623e-03
hun_dist_InterCom       -6.870765e-05
hun_dist_Intersonic     -3.549064e-03
hun_dist_Palace          2.991911e-04
hun_dist_Paramount      -1.569193e-03
hun_dist_Provideo        1.898481e-03
hun_dist_SPI            -1.146425e-03
hun_dist_SonyColumbia    6.002063e-04
hun_dist_United         -6.203283

In [410]:
x = factors_2['budget'].fillna(0)
y = factors_2['y_values'].fillna(0)
new_df = pd.DataFrame({'X':x, 'Y':y})

In [411]:
model = sm.ols('Y ~ X', data=new_df)

In [412]:
results = model.fit()

In [415]:
print('Parameters: ', results.params)
print('R2: ', results.rsquared)
print(results.summary())

('Parameters: ', Intercept    4.625360e-03
X           -6.357197e-12
dtype: float64)
('R2: ', 0.015403844417560042)
                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     6.774
Date:                Tue, 26 Jan 2016   Prob (F-statistic):            0.00957
Time:                        18:59:24   Log-Likelihood:                 1850.8
No. Observations:                 435   AIC:                            -3698.
Df Residuals:                     433   BIC:                            -3689.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------

In [414]:
results.predict(new_df['X'])

array([], dtype=float64)

In [403]:
results

In [395]:
y.shape

(435,)

In [418]:
# plt.scatter(x,y)
# plt.plot(x, results.predict(x), color='green',linewidth=3)

Please picture a beautiful nearly horizontal line at y=0.46 in size 3 and color blazing green to contrast the royal blue of my datapoints. Please forgive statsmodels for being terrible and hating me.

In [417]:
# # Import
# from sklearn import linear_model

# # Create an object
# rgr = linear_model.LinearRegression()

In [416]:
# x_values = x.values
# y_values = y.values

# # Fit the model. Takes a multidimensional numpy array
# rgr.fit(x_values,y_values)

# # Make prediction
# pred = rgr.predict(x_values)

# print("coefficient {}".format(rgr.coef_))
# print("intercept {}".format(rgr.intercept_))

# plt.scatter(x_values,y_values)
# plt.plot(x_values, rgr.predict(x), color='blue',linewidth=3)